# LDA에서 Author Topic Model

<img src="figs/atm_fig.png">

* LDA에서 저자정보(metadata)가 추가된 토픽모델.
* 문서별 단어분포를 반영하던 파라미터에, 저자별 문서분포 정보까지 추가된 형태.

<img src="figs/atm_fig2.png">

In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

import pickle

In [2]:
with open("data/cleaned_data.pk", "rb") as f:
    data = pickle.load(f)
    
data.reset_index(drop=True, inplace=True)
print(data.head())
print(data.info())

               Date User            Message
0  2017/01/01 20:32   무지  이거보면 왜 갓창정인지 알게된다
1  2017/01/01 21:40  어피치                창정헌
2  2017/01/01 22:19  어피치   라이언은 내일부터 연구실 출근
3  2017/01/01 22:20   무지          파티 하는거 아님
4  2017/01/01 22:39  프로도                헬파티
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162861 entries, 0 to 162860
Data columns (total 3 columns):
Date       162861 non-null object
User       162861 non-null object
Message    162861 non-null object
dtypes: object(3)
memory usage: 3.7+ MB
None


In [3]:
users = set(data["User"])
users

{'네오', '라이언', '무지', '어피치', '튜브', '프로도'}

In [4]:
authors = data.groupby('User')
pprint(authors.groups)
print(type(authors.groups))

{'네오': Int64Index([   273,    274,    276,    277,    342,    374,    396,   3784,
              3785,   3787,
            ...
            157425, 157427, 158059, 158060, 158062, 161067, 161068, 161074,
            161075, 161080],
           dtype='int64', length=324),
 '라이언': Int64Index([     9,     14,     15,     18,     19,     24,     25,     26,
                30,     32,
            ...
            162840, 162843, 162844, 162845, 162846, 162849, 162850, 162852,
            162854, 162860],
           dtype='int64', length=39243),
 '무지': Int64Index([     0,      3,      6,      8,     10,     11,     13,     16,
                17,     22,
            ...
            162815, 162816, 162820, 162822, 162830, 162834, 162836, 162839,
            162841, 162842],
           dtype='int64', length=47861),
 '어피치': Int64Index([     1,      2,      5,      7,     12,     20,     21,     23,
                27,     28,
            ...
            162649, 162654, 162655, 162668, 162670, 16

In [5]:
author2doc = {}

for user, index in authors.groups.items():
    author2doc[user] = list(index)
    
print(author2doc)

{'네오': [273, 274, 276, 277, 342, 374, 396, 3784, 3785, 3787, 3791, 3793, 3799, 3801, 3803, 3805, 3808, 3809, 3811, 3813, 3816, 3819, 3820, 3823, 3826, 3848, 3849, 3850, 3851, 3855, 3857, 3862, 3870, 3873, 3874, 3889, 3890, 3892, 3897, 3901, 3905, 3911, 3929, 3934, 3936, 3937, 3938, 3943, 3947, 4418, 4419, 4427, 4429, 4436, 4437, 4438, 4440, 4444, 4445, 4451, 4452, 4454, 4458, 4461, 4462, 4464, 4472, 4474, 4480, 4482, 4484, 4490, 4491, 4493, 4495, 4498, 4500, 4501, 4502, 4504, 4506, 4508, 4512, 4515, 4518, 4520, 4521, 4527, 4532, 4536, 4541, 4543, 15741, 32226, 32227, 32231, 32233, 32234, 32236, 32253, 32256, 32260, 32261, 32264, 32267, 42185, 42188, 42189, 42192, 42197, 42208, 42286, 42289, 42290, 42291, 42293, 45199, 45202, 45208, 45209, 45227, 45228, 48740, 48741, 48742, 48743, 48748, 49663, 49665, 49668, 49669, 49671, 49673, 49675, 49677, 49694, 49697, 49699, 49701, 49702, 49705, 84101, 84102, 84107, 84108, 84111, 84114, 84118, 84120, 85304, 93097, 93098, 93100, 93101, 106745, 10683

In [6]:
tokenized_data = [msg.split() for msg in list(data["Message"])]
print(tokenized_data[:10])

[['이거보면', '왜', '갓창정인지', '알게된다'], ['창정헌'], ['라이언은', '내일부터', '연구실', '출근'], ['파티', '하는거', '아님'], ['헬파티'], ['헬파이어'], ['파이어에그'], ['리마리오'], ['프로도'], ['예스']]


In [7]:
from gensim.models import AuthorTopicModel
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary, bleicorpus
from gensim.matutils import hellinger
from gensim import corpora
from tqdm import tqdm_notebook
from time import time

import os

In [8]:
# ATM에 사용할 Dictionary 만들기
if not os.path.exists('kakao(ATM)_dict'):
    dictionary = corpora.Dictionary(tokenized_data)
    dictionary.save('kakao(ATM)_dict')
    print(dictionary)
else:
    dictionary = Dictionary.load('kakao(ATM)_dict')

# ATM에 사용할 corpus 만들기
if not os.path.exists('kakao(ATM)_corpus'):
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
    corpora.BleiCorpus.serialize('kakao(ATM)_corpus', corpus)
else:
    corpus = bleicorpus.BleiCorpus('kakao(ATM)_corpus')

In [9]:
print('Number of authors: %d' % len(authors))
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

Number of authors: 6
Number of unique tokens: 114854
Number of documents: 162861


In [10]:
# dictionary함수로 만든 사전에 있는 단어 보기
print(dictionary)
for idx in dictionary:
    print(dictionary[idx])

Dictionary(114854 unique tokens: ['갓창정인지', '알게된다', '왜', '이거보면', '창정헌']...)
갓창정인지
알게된다
왜
이거보면
창정헌
내일부터
라이언은
연구실
출근
아님
파티
하는거
헬파티
헬파이어
파이어에그
리마리오
프로도
예스
호우
팟팟헌
리마리오헌
느끼헌
안그래도
오늘힘듬
대단
허리
괘안
이정도는가뿐
즐거운
첫츌근
야옹
돈많이
벌어와유
보고싶다
프로도신
그래도
다행인듯
얼굴봐서
저번에
되었다
만날때가
이제
튜브를
기대하라
다음주
굿굿
2박3일
곤지암으로
담주에
스키타러감
데려가삼
스키담
하앜
라이언
여기
좋음
너의
떠나버린
세컨과
졸업하고
닮은
아이
잇음
목소리도
비슷
심지어
다름
이름이
하지만
연수아냐
신수인씨
신수인
10학번이랑
이름똑같넼
걔는
남자
물론
Ma
굿
금결원
많음
이쁜사람
교육받는중이려나
4명임
여자
몇명
입사
25
1
6분의
간택될수있을것인가
과연
어피치는
되야함
미스코리아는
지노우
없어짐
팬더
무지
팬더버림
변하니
사랑이
어떻게
바람둥이임
나픈아이
저런
벌써
여럿울림
먹튀
당연
그건
기본임
이런마
노답
지하철
퇴근
모습으로
복귀예정
새로운
조만간
모으는줄
발견해서
사려고
엄청난
쿠션을
푼돈
중
최악
2호선
과소평가한듯
그동안
내가
2호선은
그자체임
시간대의
이프로도대마왕
진정한
그정도
되지
기운내삼
출근시간의
지옥그자체지
3개보내고
4번째탐
2명을
보며
힘내삼
끝나
언제
튜브
아심
12일넘어서
1월
만나자고함
데리러가는일은
없길
한강으로
빨리
슬픔
기다리셈
돈이
없음
알바
누가
사갈까봐
조마조마중
진심
백곰카페라도
너무
따뜻하다
복지가
연봉과
머스트
컴
제발가고싶다
담학기뒷통수
rr
고고담
넘는
상상을
초봉이엇움
건설임
라이언의
목표는
왕국
많아야함
여사원이
가야함
그리고
나와
브라질을
없다
포기할수
2월
돈모아서
한번에
건설예정
이주해서
아마존담
초봉얼마임
얼마
예상
넘는다
안했던가
오천좀
그런줄
알앗눈데
더
따뜻햇음
좀
쩌는데
어피치
역시
갓헌
말햇던
면접때
안좋게
알고보니
임원이
서강대
출신

저여자분
이날씨에뛰면
뒤지기딱좋음
쥭을듯
행군하면
가장
추운데
얼어죽는건가
동파헌
엄청
고생이
많네
어피치가
아이온임
예정
유가네
점심에
집임
쉬자
자고
잠도
그치만
위드
훔
같이잇지
아빠
엄마
하고싶은것들
해야지
오후
행군은
그런가보네
출발지
살아돌아와
고생이많구만
가리고
뎅겨
얼굴
솬이는
아이온와라
점심까지
오면
그럼나도감
닥텨
공부해
아직도
저거네
프사
1차끝나고달리게
오버워치연습좀해놔
딕텨
연습할까
연습해놔
내생각에더
도
잘함
롤보다는덜함
빠지긴할텐데
훅
질린다
한달내로
이방
일타다
렙이
백이다
질려서
워낙
익히는데시간오래걸릴수도
임구마가
호구라
두시까지
들어와
삼일전에
어제보니까
했더라
마지막으로
배치돌린다햇자나
이색꺄
내리
막판3판
위도우로
팟지찍고끝냈다
연습하고와라
한조같은놈
라인하르트
넘
딱임
한조가
한조도
멋잇는데
보이는데
프사가
허전해
드림
보자
사진좀
콜렉션있음
쓰셈
원하는거
낰
고구마
뭐냐
임구마임
맨위의거
생각보다
잘생겻네
뭐지
임팩트
그정도는
아니고
재밋구만
건데
셀카
엠디엔피에서
저거
찍엇던
돋네
추억
다운
포켓몬고
일어낫네
라이언군
바쁜하루라
피곤햇겟구만
갓구만
자러
자지
안자지
자지마
말아
보지마
맞군
딴사람
뽀오지이
히앍
야외화장실
누구냐
나다
뀨
색마둘
드셧는가
점심은
2시반에먹을예정
맛잇게
밥도
돌아가며먹어야
제맛
다양하게
달자
강의중
뭘
피방와
없쪄
현금
시간
잇긴잇구만
충전된
개이득
커먼
13분남음
확인한거
피방갓엇으니
시간이
같이만나서
하지뭐
설낳이네
낳
오타
설날
졸귀
갓더
골드찍고싶다
너희와
어울리기
위해
한다만은
약대가서도
잇으니
할수도
연습은
보지
갓헌이
무섭게
추격중
열심히함
은근
근성남
혹우
노력파헌
파헌
아놔
맥은
북미서버밖에
안되네
나랑
롤하고
싶으면
북미서버에서
해야할듯
가능
밤이나
평일
10일날
노트북을
뺄때
신촌방을
가져올건데
연구실에
내뒷자리에
모니터
연결해서
거기를
사용할건데
윈도우로
게임할까
그전에
만나자
언제볼가
기다려
그때는
만날때를
온라인에서
얘기하는것이니
집에서는
못하지
안
된다고해도
꾸

위협할
정부인을
소개시켜줄려고했는데
어쩔수
없었다
내치듯이
준희를
매정하진않았으
짊어지고가야함
평생
바빴을뿐
과
인해
전공으로
상황
추론
매정하게
무지임
준희한테더
구주영
애초에
정을떈거
그분은
행동이
매정햇음
구주영한텐
미안하다
준희보다
갈때
학원같이
길정애
커커윽
추억추억
설연고와
위슬런
김밥천국
정원이의
길정앸
드립너무하넼
마법의
볶음밥
튜브이는
이지
10줄
김밥
솬이의
모르시네
신지여
지연이
구임
뭐하고사는지
아시는분
기정이랑
엠티가서
햇대
엠티가서했대
개오랜만
들음
빅매한테
성인
사귄거아님
기정이가
망해서
우울한걸
재수
사귐
수능
재수망했을때
엠틱
대화로
몸의
적절한
조치
푸는
그엠티가
엠티가
착하네
받은듯
해피해피하군
천사가따로없군
기정이는
뭐하고사나
소식
안외애들
개궁금함
그럴수도있음
우리반도
잘모름
들을수있음
모임
여자애들
가능성이
그럴
큰듯
나가라고좀
부모님보고
부모님들도
잘안만남
맨처음부터
인연이다
질긴
몇명만나던거라
계
인천부모님들은
마미도
울
인천부천끼리
만나다가
여긴
와해됨
우승
페더러
운다
흐귷규
그런의미에서
봤는데
손주희
누군지
영어과아님
난이미
켜져있긴함
8빈
중어과
기억남
같은반
10시에
신기하게
약간
교포삘나게
맞음
사람들
처다봄
한곡남음
겜돌이
천천히
오셈
나파이썬
강의듣고있으면됨
어듐의강의
폭풍눈물
백인덕
바로옆임
코노하고
피방
무료임
강요해서
들으라고
명강의까진
아닌데
실속은
짧아서
다시봐야겟다
자바나
죠져
모르는가
스프링
아무도
할줄
죠져줘
들어는봤지만
할줄모름
그래주면
매우감사
이거다
떠나
아이온에서
딴곳으로
옮겨야할떄임
프로도이집
비어잇엇다는데
나이트
가던곳
리드좀
전문가니까
가면됨
따라만
존문가
기다리는중
오버워치좀
하던데
은은하게
자주하더라
최근접속
안가봄
한번밖에
잘암
한번밖에안가봄
튜브이와
결과가다름
더고수임
이미들어와있음
3분만기둘
다음에
특강좀
순수덩어리들이라
두분만
따라감
믿고
솬즈
별별별별
끄
애쉬
페이커담
재미있었음
아쉬운
판도있었지만
승률
50이엇는뎅
그니까아쉽
재밋음
하니
누구내
어피치찡
쏘우1
생

폭탄
한판더
붙어야지
붙자
1차전까지
50렙찍어놔
튜브은
내생각하면서
연습해라
멘티
14000일꺼다
최고뎀지가
잡으면됨
개못하네
이분은
만사천이
빠대에서
정크장인
장난임
눈쟁인
랭커고
4만임
3만은찍지
경쟁전하면
고추커
위너
필요없음
딜을
잘넣겟네
랭커가
무슨
쓸모냐
공부할테니
딜도
잘넣고
공부도
잘하고
위도우연습좀
위도우좁밥아
갔다
그는
다시간다
옵치야머
3개정도만
할줄알면됨
키우삼
다뤄야지
모든캐를
생각중임
근디
루슈는
오래하면
방법만
익혀두고
마지막까지
픽하지마
힐러
누구하지
루슈로
어쩔수없으면
때매
짱나서
어디가도
환영받음
라인해봐야지
귀여움
라인은
S라인
떙큐임
해주면
라인메신져
봇라인
탑라인
라인을
잡으셈
힙라인
주물
벽
휘두르기
궁극기가
돌진
맞지
돌진이
스킬
망치
대지분쇄
못해도
캐가
환영받는
루슈
힐켜놓고
환영
궁극기만
잘해주면되고
두개
해야겟네
떙큐
있어주면
스스로
힐
라인아나
조합이
궁극기
뽕맞은
빠따가
고추
그거할줄
해드린거
옽
정크혀
무시해
애들말은
솔저
잡대
눈쟁이라
그분이니
눈쟁이랑
차세진이
잘하는듯
세진이는
쓰렉이라
늠
서든하는거보면
이해도가
겜
센스는
겜이
안되
적응이
누굴
몰겟음
쏴야할지
정신없음
헷갈릴떈
힐러먼저
누군질
힐러가
영상보면
판단이
떄리는지도
배우니까
어떠케
복잡한
상황에서
다릉꼐
생김새가
보다보면
인식가능혀
프로도얼굴
이러는거
처럼
몽골족이
아니어도
4키로
밖에서도
보임
보면늠
데굴데굴
펑
못하겠음
평타떄매
님은
디바
고인됨
경쟁전에선
마니함
아나나
그것들에
대해서도
봐야겟굼
잼써
많아서
아나가
상황판단도
에임도
좋은애가
좋은캐임
돰
익히면
보고옴
로드호그도
다녀오삼
로드호그는
에버모어
해봣는데
어려워
궁극기는
연필깍더라
투타타타타타타
감동
눈쟁이vs차세진
본거같은데
차세진도
원챔프임
벤치프레스
안들린다
너므
쉬엇어
못버티네
밀수
잇다눈데
팔이
목표가
헐크임
존시나
한참멀었네
260임
스쿼트
300
갈길이
멀다
몇임
동양인
핸디캡
150
3번이
골반부상이후
한계
밑힌
일점오배네
그거해서뭐해
데드는
들린
바르는거지


빠뜨렷군
주어를
이가
들어가면
넣엇는지
확인할수
프로필에서
통계가보삼
최고킬수는
그런거도
나옴스
종류별로
분석가능
첨한건데
영상보고
첨으로
잼씀
5시간내에내기록뚫으면됨
모스트
정크다
깬거
저거3개빼고다함
최근에는
그럴리없음
16000되지
않앗냐
딜량을
23킬
햇을때
확인
인증샷올려봐
23킬햇을때
에서
같앗는데
경쟁전함께할
영광을드리지
사양함
프로도이랑하면
23킬하고
꼴랑
만육천이나옴
키보드배팅해서
정크랫이니깐
가능하지
좀이따가
모바일로는
전적확인안되남
확인해볼게
인증샷올려
맞아주나
애들이
저렙존이라
쉽게인정하는군
같이하면
아이디랑
팬더는
허접인데
입버워치
만나는듯
사람들을
피지컬이
렙이높아서
자꾸언밸런스됨
오빠와치
올리고와
점수나
발라줘
10대떡으로
연습하라우
이노무시키
경쟁전선
캐리해봐
부심
자신감으로
캐리하는거여
안나옴
쫄면
위도우
이런거하면
그순간만큼은
내거
파인이다
하면서때려야
다른애들이
팟지뜸
1씩
1인분
5분에
메이코패스나하는넘이
같은뎅
잘하는거
더잘함
경쟁전에도움되는걸
자리야해줫자나
졌잖아
궁연계
디바랑
지리지않앗읍
똥싸고
탈주햇자나
한넘
캐리해야지
그거까진안됨
서브힐러
걔말고도
또있었음
사람아닌애가
힐러나하지색갸
딜러하고싶은데
라서
픽들이
자리야한거아녀
메르시
숙달함
소르져
놀란다
세상이
메이
한조랑
3만딜넣고와
솔져로
그럼인정
아나해주는거
어절수없이
젠야타나
왬
무지정도하면
벽타기
무의미하다
딜량
떄려도
방벽에
받기
싫어
아그래
눈쟁이한태
딜량보다
테
따지셈
처치로
딜딸됨
방벽도
싶음
인정받고
바스로
기억안나는데
처치는
까봐
로드호그가
2만정도
실질딜은
은따는
그판이
꿀잼이엇는데
쨋든
개쩔었음
궁연계도
임쟁이가자
옵치다운겜을했다
02
첫날에
돌리게
셤끝나고
연습많이해놔
둘쨋날엔
어렵다
탱커가
탱커를연습해라
04
이런식이면
라인이오피인데
자리야랑
데뷔할듯
프로로
라인이
운영이
갖고놀수있음
저렙존에선
낮아
80안될껄
레벨이랑
상관없음
렙
고정도까지
연습하고와
점수로만
티어
2천임
티어점수가
기대가
1800대만
만들어오면
캐리해드림
3400이엇지
3

...

좀비헌
자축
어디에도2네
하루살이
넘쳐흘러
쉬워보이는데
어디에도보단
39초
안나와씀
유튭에
고음앨범
마지막이라고
고음가득
circular
op1
콘서트
물어보셈
세시반이라고함
드실분
비울순없어서
돌아왔다가
다시가야함
도르마무
세스르마무
멘티들
소망에
한해
합격하게
기도함
특정인물
있구만
이뇨속
붙어야할
사연이
그런애들은
여자3
4명은
붙을거같아
김동준
진행하고
동준찡
동준
송이
수빈
유빈
한나
키운애들
신촌팀임
14회임
좋네좋아
잘한거같다
힘들엇지만
감사드려요
값지게
느껴집니다
도와주셔서
두달여간
많이많이
멘토님께
멘토님이
받으세요🐖🐖
스터디한
시간들은
응원해주시고
힘낼수있었어요
카리튜브
걷는건가
선생님늬
사망예정
미연시
닥춋
밤멤버니까
솬이죠져노
어딜가임마
예정보다
저녁먹을때
가있어야디
내마음속❤️
있는걸💜
부른다면
엠멕꺼
들어봐야함
안올라올듯
이삼일
운명
피하고싶어
오랜만인군
올라와도
올라올듯
구라핑
안인듯
이크에크
몇시에온다고
누워있을게
사러갈까
고민돼서
느린지
모르겠어서
실험돌려놓고
최적화좀
갔다올게
저녁먹을즈음
느릴이유가
이론적으로
저녁뭐먹음
요거
먹을거면
오는게빠름
알려준
어느샌가
너희에게
29라고
구린거냐
원자번호
가는증
화곡역임
하루살이가
넘쳐흘러는
맞춰갈게
서울대입구
구쯤
구르처릉
줄서기
일등할듯
서울대입구면
신림
한참남음
그게그거임
6시즘도착
빼른데
몇번문임
한개뿐이오
다음정거장에
문열리면
들어오시오
열테니
5전
04Jul
74라니
노리고
탄건가
69로
74로
도척
많다가
타야하나
보내버리기
문래임
문래문래문래
이거타셈
영등포구철
빠진사이
엠멕에
오르내림
녹음해서보내주삼
기다리는사람
1분후에
양지로
양지
양지차돌
시간등
데릴러갈게
이뇬아
합숙하냐
델러가려고
얼굴못봐서매우아쉬움
짐싸야해서
가까우니
자주좀모여
들어가는중임
숙숙사
속속사
갈곳
허리근육
못내리고
가버려서
이태원까지
빽
막차탈수잇녀
커플이싸운다
막차는
라이언리와
밤샘할뻔
취소될수도있음
테이컬
사진가오
잠시만기다려봐
좈졐줔짘
안온자
벌이다
간나쉥퀴들
노래녹음이나올려
샹간나들
코노안갔어


단계엿음
들어오려고
끝나도
피자몰에서
치킨뷔페는
아는가보오
배당
5000원
접엇는데
5천원
충전된거
남아잇음
따라와보실
저녁먹으면서
분석좀
벌어드림
캔커피값
프로토의
나온겐가
플토임
꾼임
아침까지
적중률
베팅
신내림
뭘걸든
금액을
일상생활
내는거냐
풀라고
띠꺼움
관련도
상관없고
생활이랑
인적성과
거르려고
잘풀면
문제들
상관없는
인성이랑도
부문
썻음
Av부문
방화벽해킹전문
썼음
언어지능
연구소라서
나는구만
에이아이
AI임
머신러닝쪽이니까
연구분야가
모든직군이
Engineer임
기도좀
축하해줘
도둑놈임
만나는데뭐
늙는
처지에
개잼씀
23이면
1학년인가
썰풀이
된다묜눙
갈수잇도롯
이번응
종사자라
직종
두경기네
한경기가
배당이
32배임
걸음
16만원이네
16만원임
13200원정도
못햇네
하긴햇으나
와이프라면
투자일수도
기도하러감
여성이신듯
저버리는중
취준때매
찬스인데
들려주러와
들려주러
썰을
해주고프나
콜미베이비
내일정도
아12시
지낫구나
전화하기전에
연락만
정해들을게
통화내용은
콩그레츄환
하홯
올라오시는가
가야되고
놀기도
해야할거같아서
방학에도
솬이라
MT환
기모뀨
춘곤뀨
꽃을
주고갔는데
데이트부터
받아들여야함
날짜부터
주혜임
큰일났네
전화전에
주혜가
후보가
미드아님
인줄알았으나
거절했고
친해진거였는데
연락와서
이것저것하다가
꽃을주고
준거겠지
밥약을
고민인듯
기독교인이라
신실한
버겁긴란데
은구T
라이언고
요약문써서제출해
간나의
논개전략
에이미의
실검에도
안뜬걸
지나갈수도
기사는
나오는중
마약
지려버려따
스릴러네
하이라이트라도
이른기상
늦은취침
시청후
밤샘하루
챙겨보삼
두골
뚫었다
수강신청보더
의약품
과목일세
건강보험
어쩌구
지려씀
티켓팅
아이맥스
요기요
용아맥
보겟네
골도
봣잖슴
양아치인게
시작이라는데
연습삼아
쓰레기들
그렇다고하더라
바켜신
티켓도
효신이에게
뺏길듯
콘서트가면
점으로
점효신임
5초올리고
싸겟군
로딩중
다운완료
표정임
들이대면
뽀뽀해버리면됨
어찌할지를
혼내줘야지
혼내주자
혼인신고로
마찰계수
줄여주려고
핸드크림을
셀프일때
향기나는
향기나겟네
밤꽃향
다시

수북
다크쳐크
형님껀
아니었다
허접이었다
게이솔져
프로도솔져
165복귀
하면가야지
1번만
뽑아보면
뭘뽑아
데드임마
80은
악력때매
70도전해봤는데
악력털려서안되더라
얼터네이트그립으로
얼터네이트가
반대방향으로
자극잘됨
방지해
악력
얼터그립은
풀리는거만
데드랑
당길수
쥐어주다보니
파워있게
오느
가슴조지는날인데
데드만
추가해보겠음
전신이라
밴치하다
저세상갈뻔
고중량한다고
혼자할땐
프리로할때
안깝쳐야할즛
안전바
그런거없던데
외국산이
국산에
무리안해야할듯
가벼웠으니
다행이지
무거웠으면
생각보다는
쥭는듯
휴우가같은놈
심심할
AVideo네
시청가능
보고있다가
안잡혀감
보게될듯
교수님도
재생중
컨셉뭔지
미친개웃기네
수업시간
야한건
전혀없는데
이해가능한
수업시간은
안웃을자신있으면
장담하는데
30초뒤에
위기온다
이따봐야겟다
보고싶
익숙한분들
효과직빵
차이점이하나있음
함께옴
지노우도
가득임
배터짐
고기듬뿍
단백질보충
수치러운데
줌바댄스
줌마댄스는
초기화
우수한
10점에서
17점으로
진화했다
고민했던것들인데
확인하는게
상당히귀찮네
맞았는데
했을때
15점인가
나는이프로도이다
핸드폰과
어려웠다
딸기
바닐라와
아이스크림도
핸드폰인데
맥북은
포기할수없다
변경어때
달이
해라니
선택지에
있었으면했다
7개는
공감능력이
부족하군
7점
튜브이일수도
해벌게
해벌
15점이네
16점
떡중에
케익하고
나의것
만듦
만들어오삼
떡은
치는거지
19개
맞췄는데
장미도
틀리네
뭘틀림
틀린건
위기였고
장미가
위기였는데
그게뭐지
맞출수가
없는거였음
제설
솔지깋
그생각했는데
2019년이라서
가능할줄
뭐틀림
김네오으로
주스아님
김네오
재도전할때
이용해야지
참치마요가
이차는
두명이라
10개이상
맞출자신없음
빙의하면
저새끼는
열두개까진
12개
운좋아야
개빡치게한다
고득점임
아는사람
튜브오빠를
세컨담
시켰을때
여자친구에게
남친으로
이녁석들
만들어라
친히해드림
박성민
선생님임
어케만들어
버튼도있음
2019
‼️👇👇👇👇👇👇‼️
시작하세요
👸🤴️
🤜
75퍼만맞춰라
취향특이하네
풀어보러
15점
얼른자라
5년째
쉬다오삼
국방의
다하러

준비하면됨
아웃라인
가르침좀
사귀는것만
안나가니까
학교밖에
소개내놔
큐피트환
튜브이밖에없어
교사들
동료
안좋아할듯
전화중이구
물어봣는데
생각나는
하시네
올거니
21짤이라도
갠찬다믄
물어는
22살들이랑
다니긴함
빨리서
8살이라니
입학하고
지나서야
완벽하네
음란패설을
어렵겟지
이해하기란
음란
이해해야지
여자들도하는데
음란패설은
도덕성
싶은걸
자연스러운거라
아는사람이나
범죄거나
괜찮다고생각
미친소리만
몇갠데
야한말이야
도닦는
스님도
당연한거아닌가
동물이고
인간이고
신선도
정신수련해서
본능적인거임
안생기게
도와주던가
2년을
싸워봐서
텄지
고딩때부터
해오던건디뭐
남톡방에서
올리는거는
짤을
싸우지는
않앗고
주제를
경험에
야한얘기를
자기의
발언만
성추행적
안나가면
O
관계중에
빠진적이
예를
콘돔이
하는것과
것과는
여성의
별개이다
일반인이면
야한짤이
그런게아니라
야한짤도
시가수
갠찬자너유
무대보고
얘긴
공유하면서
올리는건
이상한거고
이상한거겟지
그러려는
매체에
생각함스
야동추천은
아닌거고
갠찬은디
성범죄도
왼손한테
죄짓는거지
이상햇다
전화를
아니라고만
말하다가
이해하고
전화끊은듯
강다니엘
돌려보는건
스셧
안되남
되는거라고
추천하면서
공유하는건
노골적이라
그러함쓰
의견은
노골적이긴하지
머쓱쓰
성추행은
여자입장에선
공론화햇다는게
별로일수도
이해를
잇겟군
하게는
이해는
그들이
이해하기
힘들단거
그랫다면
별로라는게
이해는됨
충분쓰
야동에
우리같이
절여져있지않아서
골자엿으니
건전한겨
된다아니여
성추행만
성희롱
아니믐
그러하다고
남성이
이해해달라
내가옴
어찌생각하심
일부
이상할것같지만
되는정도
애인이
인것같고
보든
안보든
야동도
몰카나
범죄만
가능하다고
추천이야
할수있고
그러췌
생각하는데
성생활이
얘기하는걸
별상관없는데
야동얘기하는건
남한테
공유햇엇다
느끼심
이상함을
가치관의
차이니까
야한짤이냐에
싫어할수도있음
안맞는거긴한데
짤이던간
어려워함
차이인거지
받아들엿다
얘기면
정준영아니냐
짤은
그러하나
쓰레긴
뭔말인지는
재활용은
얘기한건아님
아까말햇듯이
많이싸울텐데
대화하는것과
오엑스
친구

이기능
저기능
그냥하셈
생각바꼇다고하삼
새퀴
혼이담긴
날라가붕게
기대되는구만
전담
타짜들
회계사임


In [15]:
# 사람이 이해할 수 있는 형태로 코퍼스 사전 재구성 해보기 (term-frequency)
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus]

[[('갓창정인지', 1.0), ('알게된다', 1.0), ('왜', 1.0), ('이거보면', 1.0)],
 [('창정헌', 1.0)],
 [('내일부터', 1.0), ('라이언은', 1.0), ('연구실', 1.0), ('출근', 1.0)],
 [('아님', 1.0), ('파티', 1.0), ('하는거', 1.0)],
 [('헬파티', 1.0)],
 [('헬파이어', 1.0)],
 [('파이어에그', 1.0)],
 [('리마리오', 1.0)],
 [('프로도', 1.0)],
 [('예스', 1.0)],
 [('호우', 1.0)],
 [('팟팟헌', 1.0)],
 [('리마리오헌', 1.0)],
 [('느끼헌', 1.0)],
 [('안그래도', 1.0)],
 [('오늘힘듬', 1.0)],
 [('대단', 1.0)],
 [('허리', 1.0)],
 [('괘안', 1.0)],
 [('이정도는가뿐', 1.0)],
 [('즐거운', 1.0), ('첫츌근', 1.0)],
 [('야옹', 1.0)],
 [('돈많이', 1.0), ('벌어와유', 1.0)],
 [('보고싶다', 1.0), ('프로도신', 1.0)],
 [('그래도', 1.0), ('다행인듯', 1.0), ('얼굴봐서', 1.0), ('저번에', 1.0)],
 [('되었다', 1.0), ('만날때가', 1.0), ('이제', 1.0), ('튜브를', 1.0)],
 [('기대하라', 1.0), ('다음주', 1.0)],
 [('굿굿', 1.0)],
 [('2박3일', 1.0), ('곤지암으로', 1.0), ('담주에', 1.0), ('스키타러감', 1.0)],
 [('데려가삼', 1.0), ('스키담', 1.0)],
 [('하앜', 1.0)],
 [('라이언', 1.0), ('여기', 1.0)],
 [('좋음', 1.0)],
 [('너의', 1.0), ('떠나버린', 1.0), ('세컨과', 1.0), ('졸업하고', 1.0)],
 [('닮은', 1.0), ('아이', 1.0), ('잇음', 1.0)],
 

In [12]:
# Author Topic Model 실행
if not os.path.exists("kakao(ATM)_model"):
    model = AuthorTopicModel(corpus=corpus, num_topics=4, id2word=dictionary.id2token, \
                author2doc=author2doc, passes=10)
    model.save('kakao(ATM)_model')
else:
    model = AuthorTopicModel.load("kakao(ATM)_model")

In [13]:
model.show_topic(0)

[('닥쳐', 0.00913352474795045),
 ('오우', 0.006077150842656581),
 ('핡', 0.005814291204809277),
 ('넌', 0.004275330190804228),
 ('그게', 0.003936329056179983),
 ('찍', 0.003518902685383488),
 ('됨', 0.0033362068393125244),
 ('일단', 0.0031497496609154147),
 ('가서', 0.0030418551056656984),
 ('올', 0.002905564005796451)]